In [1]:
import imaplib
import email
from email.header import decode_header
import os
import webbrowser
from transformers import pipeline

In [2]:
import config

# Use the variables from config file
mail = config.username
password = config.password

imap = imaplib.IMAP4_SSL("imap.gmail.com")  # establish connection

imap.login(mail, password)  # login

response, mailboxes = imap.list()

for mailbox in mailboxes:
    print(mailbox.decode())

status, messages = imap.select("INBOX")  # select inbox

(\HasNoChildren) "/" "INBOX"
(\HasChildren \Noselect) "/" "[Gmail]"
(\HasNoChildren \Sent) "/" "[Gmail]/G&APY-nderilmi&AV8- Postalar"
(\HasNoChildren \Junk) "/" "[Gmail]/Spam"
(\Drafts \HasNoChildren) "/" "[Gmail]/Taslaklar"
(\All \HasNoChildren) "/" "[Gmail]/T&APw-m Postalar"
(\Flagged \HasNoChildren) "/" "[Gmail]/Y&ATE-ld&ATE-zl&ATE-"
(\HasNoChildren \Trash) "/" "[Gmail]/&AMcA9g-p kutusu"
(\HasNoChildren \Important) "/" "[Gmail]/&ANY-nemli"


In [3]:
numOfMessages = int(messages[0]) # get number of messages
 
print(numOfMessages)

def clean(text):
    # clean text for creating a folder
    return "".join(c if c.isalnum() else "_" for c in text)
 
def obtain_header(msg):
    # decode the email subject
    subject, encoding = decode_header(msg["Subject"])[0]
    if isinstance(subject, bytes):
        subject = subject.decode(encoding)
 
    # decode email sender
    From, encoding = decode_header(msg.get("From"))[0]
    if isinstance(From, bytes):
        From = From.decode(encoding)
 
    print("Subject:", subject)
    print("From:", From)
    return subject, From

1754


In [4]:
def download_attachment(part):
    # download attachment
    filename = part.get_filename()
 
    if filename:
        folder_name = clean(subject)
        if not os.path.isdir(folder_name):
            # make a folder for this email (named after the subject)
            os.mkdir(folder_name)
            filepath = os.path.join(folder_name, filename)
            # download attachment and save it
            open(filepath, "wb").write(part.get_payload(decode=True))
 

In [5]:
import email
import re
from email.header import decode_header

for i in range(numOfMessages, 0, -1):
    res, msg = imap.fetch(str(i), "(RFC822)")  # fetches the email using it's ID     
        # Assuming 'msg' is your list of responses
    for response in msg:
        if isinstance(response, tuple):
            raw_email = response[1]
            email_message = email.message_from_bytes(raw_email)
    
            if email_message.is_multipart():
                for part in email_message.walk():
                    if part.get_content_type() == "text/plain":
                        charset = part.get_content_charset()
                        if charset is None:
                            charset = 'utf-8'
                        body = part.get_payload(decode=True)
                        body = body.decode(charset, errors='replace')
                        # Remove URLs
                        body = re.sub(r'http\S+|www.\S+', '', body, flags=re.MULTILINE)
            else:
                if email_message.get_content_type() == "text/plain":
                    charset = email_message.get_content_charset()
                    if charset is None:
                        charset = 'utf-8'
                    body = email_message.get_payload(decode=True)
                    body = body.decode(charset, errors='replace')
                    # Remove URLs
                    body = re.sub(r'http\S+|www.\S+', '', body, flags=re.MULTILINE)
    
    path = f"own_mails\\email{i}.txt"

    with open(path, 'w', encoding='utf-8') as f:
        # Write the content of the variable to the file
        f.write(body)
